In [ ]:
from typing import Type
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool
from langchain_core.messages.ai import AIMessage
from langchain.agents import create_agent
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun, DuckDuckGoSearchResults
from langchain_community.document_loaders.web_base import WebBaseLoader
from pydantic import BaseModel, Field
import os

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini",
)


class SearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")


class DuckDuckGoScrapeToolArgsSchema(BaseModel):
    urls: list[str] = Field(description="The query you will search for")


class WikipediaSearchTool(BaseTool):
    name: str = "WikipediaSearchTool"
    description: str = """
    Use this tool to search the query from Wikipedia.
    It takes a query as an argument.
    """
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        wp = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wp.run(query)


class DuckDuckGoSearchTool(BaseTool):
    name: str = "DuckDuckGoSearchTool"
    description: str = """
    Use this tool to search the query from DuckDuckGo.
    It takes a query as an argument.
    """
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        result = ddg.run(query)
        # 획득한 결과에서 url을 뽑아내서 리스트로 채우는 코드
        urls = [chunk.split("]")[0] for chunk in result.split("link: ")][1:]
        return urls


class DuckDuckGoScrapeTool(BaseTool):
    name: str = "DuckDuckGoScrapeTool"
    description: str = """
    Use this tool to get the URL when searching with DuckDuckGo.
    It takes a list of URL as an argument.
    """
    args_schema: Type[DuckDuckGoScrapeToolArgsSchema] = DuckDuckGoScrapeToolArgsSchema

    def _run(self, urls):
        loader = WebBaseLoader(urls)
        docs = loader.load()
        return docs


agent = create_agent(
    model=llm,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
        DuckDuckGoScrapeTool(),
    ],
)

content = "Research about the XZ backdoor"

result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": content,
            }
        ]
    }
)

In [21]:
base_dir = "./.cache/agent_results/"
if not os.path.exists("./cache"):
    os.mkdir("./cache")
if not os.path.exists("./.cache/agent_results/"):
    os.mkdir("./.cache/agent_results/")
with open(f"{base_dir}result.txt", "w", encoding="utf-8") as f:
    f.write(
        f"Input: {result["messages"][0].content}\n\nOutput:\n{result["messages"][2].content}"
    )